In [ ]:
class config:
    MAX_LEN = 165
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 8
    BASE_MODEL_PATH = "vinai/phobert-base-v2"
    # MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/ner vietnam"
    TRAINING_FILE = "/content/drive/MyDrive/Colab Notebooks/vietnam ner/train_2016_final.csv"
    TESTING_FILE = "/content/drive/MyDrive/Colab Notebooks/vietnam ner/test_2016_final.csv"
    TOKENIZER = transformers.AutoTokenizer.from_pretrained(BASE_MODEL_PATH, do_lower_case=True)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv(config.TRAINING_FILE,encoding='utf-16')
train_data.head()

,Sentences,Word,POS1,TAG
0,0,Đó,P,O
1,0,là,V,O
2,0,con,Nc,O
3,0,đường,N,O
4,0,biển,N,O


In [ ]:
test_data = pd.read_csv(config.TESTING_FILE, encoding='utf-16')
test_data.head()

,Sentences,Word,POS1,TAG
0,0,Chị,Nc,O
1,0,Minh,Np,B-PER
2,0,ôm,V,O
3,0,đứa,Nc,O
4,0,con_gái,N,O


In [ ]:
class EntityDataset:
    def __init__(self, texts, pos, tags):
        self.texts = texts
        self.pos = pos
        self.tags = tags
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        pos = self.pos[item]
        tags = self.tags[item]

        ids = []
        target_pos = []
        target_tag =[]
        mask=[]
        token_type_ids=[]


        inputs = config.TOKENIZER.encode(
                text,
                add_special_tokens=False
            )
        input_len = len(inputs)
        padding_len =config.MAX_LEN - input_len
        ids.extend(inputs + [0]*padding_len )
        target_pos.extend(pos+ [2]*padding_len)
        target_tag.extend(tags + [8]*padding_len)

        mask.extend([1] * input_len + [0]*padding_len)
        token_type_ids.extend([0]*config.MAX_LEN)

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_pos": torch.tensor(target_pos, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

In [ ]:
def process_data(data_path):
    df = pd.read_csv(data_path,encoding='utf-16')

    df.dropna(inplace=True)

    enc_tag = preprocessing.LabelEncoder()
    enc_pos = preprocessing.LabelEncoder()
    df.loc[:, "TAG"] = enc_tag.fit_transform(df["TAG"])
    df.loc[:, "POS1"] = enc_pos.fit_transform(df["POS1"])

    sentences = df.groupby("Sentences")["Word"].apply(list).values
    tag = df.groupby("Sentences")["TAG"].apply(list).values
    pos= df.groupby("Sentences")["POS1"].apply(list).values
    df.loc[:, "OriginalLabel"] = enc_tag.inverse_transform(df["TAG"])
    df.loc[:, "OriginalPOS"] = enc_pos.inverse_transform(df["POS1"])


    label_mapping = df[["TAG", "OriginalLabel"]].drop_duplicates().sort_values("TAG")
    pos_mapping = df[["POS1", "OriginalPOS"]].drop_duplicates().sort_values("POS1")




    return sentences, tag, enc_tag,pos,enc_pos

In [ ]:
train_sentences, train_tag, enc_tag_train,pos_tag, enc_pos = process_data(config.TRAINING_FILE)

In [ ]:
test_sentences, test_tag, enc_tag_test,pos_tag_test, enc_pos_test = process_data(config.TESTING_FILE)

<ipython-input-11-c2eb7c547444>:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "TAG"] = enc_tag.fit_transform(df["TAG"])
<ipython-input-11-c2eb7c547444>:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "POS1"] = enc_pos.fit_transform(df["POS1"])


In [ ]:
train_dataset = EntityDataset(texts=train_sentences, pos=pos_tag, tags=train_tag)

In [ ]:
test_dataset = EntityDataset(texts=test_sentences, pos=pos_tag_test, tags=test_tag)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.TRAIN_BATCH_SIZE, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.VALID_BATCH_SIZE, num_workers=1)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def train_fn(data_loader, model, optimizer,optimizer1, device, scheduler):
    model.train()
    pos_losses = []
    ner_losses = []

    predicted=[]
    real=[]
    predicted_pos=[]
    real_pos=[]



    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        optimizer1.zero_grad()
        output_tag, output_pos, loss_tag,loss = model(**data)
        # pro.append(p)




        loss.backward(retain_graph=True)
        loss_tag.backward(retain_graph=True)

        for sublist_a, sublist_b in zip(output_tag, data["target_tag"]):
          n = len(sublist_a)
          first_n_values = sublist_b[:n]

          real.append(first_n_values)
          predicted.append(sublist_a)

        for sublist_a, sublist_b in zip(output_pos, data["target_pos"]):
          n = len(sublist_a)
          first_n_values = sublist_b[:n]

          real_pos.append(first_n_values)
          predicted_pos.append(sublist_a)


        optimizer.step()
        optimizer1.step()
        scheduler.step()

        pos_losses.append(loss.item())
        ner_losses.append(loss_tag.item())

    avg_pos_loss = sum(pos_losses) / len(pos_losses)
    avg_ner_loss = sum(ner_losses) / len(ner_losses)

    y_true_tag = torch.tensor([label for sublist in real for label in sublist]).cpu()
    del real
    y_pred_tag = torch.tensor([label for sublist in predicted for label in sublist]).cpu()
    del predicted
    f1_tag = f1_score(y_true_tag, y_pred_tag, average='micro')

    y_true_pos = torch.tensor([label for sublist in real_pos for label in sublist]).cpu()
    del real_pos
    y_pred_pos = torch.tensor([label for sublist in predicted_pos for label in sublist]).cpu()
    del predicted_pos
    f1_pos = f1_score(y_true_pos, y_pred_pos, average='micro')


    return avg_pos_loss, avg_ner_loss,f1_pos,f1_tag

def eval_fn(data_loader, model, device):
    model.eval()
    pos_losses = []
    ner_losses = []

    predicted=[]
    real=[]
    predicted_pos=[]
    real_pos=[]



    with torch.no_grad():
      for data in tqdm(data_loader, total=len(data_loader)):
          for k, v in data.items():
              data[k] = v.to(device)

          output_tag, output_pos, loss_tag,loss = model(**data)

          pos_losses.append(loss.item())
          ner_losses.append(loss_tag.item())
          for sublist_a, sublist_b in zip(output_tag, data["target_tag"]):
            n = len(sublist_a)
            first_n_values = sublist_b[:n]

            real.append(first_n_values)
            predicted.append(sublist_a)

          for sublist_a, sublist_b in zip(output_pos, data["target_pos"]):
            n = len(sublist_a)
            first_n_values = sublist_b[:n]

            real_pos.append(first_n_values)
            predicted_pos.append(sublist_a)

      avg_pos_loss = sum(pos_losses) / len(pos_losses)
      avg_ner_loss = sum(ner_losses) / len(ner_losses)

      y_true_tag = torch.tensor([label for sublist in real for label in sublist]).cpu()
      del real
      y_pred_tag = torch.tensor([label for sublist in predicted for label in sublist]).cpu()
      del predicted
      f1_tag = f1_score(y_true_tag, y_pred_tag, average='micro')

      y_true_pos = torch.tensor([label for sublist in real_pos for label in sublist]).cpu()
      del real_pos
      y_pred_pos = torch.tensor([label for sublist in predicted_pos for label in sublist]).cpu()
      del predicted_pos
      f1_pos = f1_score(y_true_pos, y_pred_pos, average='micro')


    return avg_pos_loss, avg_ner_loss,f1_pos,f1_tag


In [ ]:
import torch.nn.functional as F
log_soft = F.log_softmax

In [ ]:
class EntityModel(nn.Module):
    def __init__(self, num_tag, num_pos):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.num_pos = num_pos
        self.phobert = transformers.AutoModel.from_pretrained(config.BASE_MODEL_PATH,return_dict=False)
        # for param in self.phobert.parameters():
        #     param.requires_grad = False
        self.bert_drop_1 = nn.Dropout(0.3)
        self.bert_drop_2 = nn.Dropout(0.3)
        self.out_tag = nn.Linear(768, self.num_tag)
        self.out_pos = nn.Linear(768, self.num_pos)
        self.crf = CRF(num_tag, batch_first = True)
        self.crf_pos = CRF(num_pos, batch_first = True)
    def forward(
        self,
        ids,
        mask,
        token_type_ids,
        target_pos,
        target_tag
    ):
        o1, _ = self.phobert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )

        bo_tag = self.bert_drop_1(o1)
        bo_pos = self.bert_drop_2(o1)

        tag = self.out_tag(bo_tag)
        pos = self.out_pos(bo_pos)



        if target_tag is not None:
            loss_tag = -self.crf(log_soft(tag, 2), target_tag, mask=mask.type(torch.uint8), reduction='mean')
            prediction = self.crf.decode(tag, mask=mask.type(torch.uint8))


        else:
            prediction = self.crf.decode(tag, mask=mask.type(torch.uint8))
            # return prediction

        if target_pos is not None:
            loss_pos = -self.crf_pos(log_soft(pos, 2), target_pos, mask=mask.type(torch.uint8), reduction='mean')
            pos_prediction = self.crf_pos.decode(pos, mask=mask.type(torch.uint8))


        else:
            pos_prediction = self.crf_pos.decode(pos, mask=mask.type(torch.uint8))


        return prediction, pos_prediction,loss_tag,loss_pos

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
num_tag = len(list(enc_tag_train.classes_))
num_pos = len(list(enc_pos.classes_))

In [ ]:
model = EntityModel(num_tag=9, num_pos=20)
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EntityModel(
  (phobert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [ ]:
param_optimizer = list(model.named_parameters())
# no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
# no_decay = ['bias', 'gamma', 'beta','LayerNorm.weight','LayerNorm.bias']
no_decay = ['bias', 'gamma', 'beta',"LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(
                nd in n for nd in no_decay
            )
        ],
        "weight_decay": 0.001,
    }

]

num_train_steps = int(
    len(train_sentences) / config.TRAIN_BATCH_SIZE * 10
)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
optimizer1 = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)

best_loss = np.inf
# best_acc = 0.0
for epoch in range(10):
    train_pos_losses, train_ner_losses,f1_pos_train,f1_tag_train = train_fn(
        train_data_loader,
        model,
        optimizer,
        optimizer1,
        device,
        scheduler
    )

    test_pos_losses, test_ner_losses,f1_pos_test,f1_tag_test = eval_fn(
        test_data_loader,
        model,
        device
    )

    print(f"Train POS Loss = {train_pos_losses:.4f}")
    print(f"Train NER Loss = {train_ner_losses:.4f}")
    print(f"Valid POS Loss = {test_pos_losses:.4f}")
    print(f"Valid NER Loss = {test_ner_losses:.4f}")

    print(f"Train POS F1 = {f1_pos_train:.4f}")
    print(f"Train NER f1 = {f1_tag_train:.4f}")
    print(f"Valid POS f1 = {f1_pos_test:.4f}")
    print(f"Valid NER f1 = {f1_tag_test:.4f}")


    test=test_pos_losses + test_ner_losses
    if test < best_loss:
        # torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/test_model/best_model_crf2.pth')
        print("Save model.")
        best_loss = test
# torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/test_model/best_model_crf3.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/425 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running

KeyboardInterrupt: ignored